In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from scipy.stats import zscore
import warnings
warnings.filterwarnings("ignore")
sns.set(color_codes=True)

In [ ]:
# load the dataset
df = pd.read_csv("../input/santander-customer-satisfaction/train.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
nulls = df.isnull().sum()
print(nulls[nulls > 0])

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.isna().sum().sum()

In [ ]:
nas = df.isna().sum()
print(nas[nas > 0])

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
coltypes = df.dtypes
print(coltypes)

In [ ]:
df['TARGET'].value_counts().plot.pie(autopct='%1.2f%%',shadow=True)

In [ ]:
dfTarget_0=df[df['TARGET']==0]
dfTarget_1=df[df['TARGET']==1]
print(dfTarget_0.shape)
print(dfTarget_1.shape)

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
dfTarget_1_up_sampled = resample(dfTarget_1, 
                                 replace=True, 
                                 n_samples=df['TARGET'].value_counts()[0],# to match majority class sample count
                                 random_state=1) 
print(dfTarget_1_up_sampled.shape)

In [ ]:
df_upsampled = pd.concat([dfTarget_0, dfTarget_1_up_sampled])
print(df_upsampled.shape)

In [ ]:
X =  df_upsampled.drop(['TARGET', 'ID'],axis=1)
Y = df_upsampled['TARGET']

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
#oversample = SMOTE()
#X, Y = oversample.fit_resample(X, Y)

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
scaler = StandardScaler()
XScaled = scaler.fit_transform(X)

In [ ]:
pca = PCA()
pca.fit(XScaled)

In [ ]:
len(pca.explained_variance_)

In [ ]:
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
y = np.cumsum(pca.explained_variance_ratio_ * 100)
x = np.arange(1,len(pca.explained_variance_)+1)
plt.plot(x,y)
plt.show()

In [ ]:
cumsum = np.cumsum(pca.explained_variance_ratio_)

d = np.argmax(cumsum >= 0.995) + 1
d

In [ ]:
pca = PCA(n_components=158)
pca.fit(XScaled)

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
X_pca = pca.fit_transform(XScaled)

In [ ]:
pca_df = pd.DataFrame(data = X_pca)
pca_df.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.3, random_state=100)


In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
randomForest = RandomForestClassifier(n_estimators = 200)
randomForest = randomForest.fit(x_train, y_train)

In [ ]:
score = randomForest.score(x_test , y_test)
print(score)

In [ ]:
y_predict = randomForest.predict(x_test)

In [ ]:
print(confusion_matrix(y_test,y_predict, labels=[0, 1]))
print(classification_report(y_test, y_predict))

In [ ]:
cm=metrics.confusion_matrix(y_test,y_predict, labels=[0, 1])

df_cm = pd.DataFrame(cm, index = [i for i in ["0","1"]],
                  columns = [i for i in ["Predict 0","Predict 1"]])
plt.figure(figsize = (7,5))
sns.heatmap(df_cm, annot=True, fmt='g')

In [ ]:
test_df = pd.read_csv("/kaggle/input/santander-customer-satisfaction/test.csv")
test_df

In [ ]:
resultDataSet = pd.DataFrame()
resultDataSet['ID'] = test_df['ID']
resultDataSet.head()

In [ ]:
test_df = test_df.drop(['ID'], axis = 1)

In [ ]:
test_df.isnull().sum().sum()

In [ ]:
test_df.isna().sum().sum()

In [ ]:
test_Scaled = scaler.fit_transform(test_df)

In [ ]:
#pca = PCA(n_components=150)
#pca.fit(test_Scaled)
test_pca = pca.fit_transform(test_Scaled)

In [ ]:
testpca_df = pd.DataFrame(data = test_pca)
testpca_df.head()

In [ ]:
test_predict = randomForest.predict(testpca_df)

In [ ]:
len(testpca_df)

In [ ]:
len(test_predict)


In [ ]:
resultDataSet['TARGET'] = test_predict
resultDataSet

In [ ]:
resultDataSet.to_csv('submission.csv', index=False)